# Purpose

If you want to end up doing Monte Carlo simulations, you will need a way to generate random numbers with different distributions

# Imports

In [1]:
import numpy as np

## Functions

In [2]:
def sn_random_numbers(shape, antithetic=True, moment_matching=True, fixed_seed=False):
        '''
        Returns array of shape shape with (pseudo)random numbers that are standard normally distributed.
        
        Parameters
        ==========
        shape : tuple (o, n, m)
            generation of array with shape (o, n, m)
        antithetic : Boolean
            generation of antithetic variates
        moment_matching : Boolean
            matching of first and second moments
        fixed_seed : Boolean
            flag to fix the seed
            
        Results
        =======
        ran : (o, n, m) array of (pseudo)random numbers
        '''
        if fixed_seed:
            np.random.seed(1000)
        if antithetic:
            ran = np.random.standard_normal((shape[0], shape[1], shape[2] / 2))
            ran = np.concatenate((ran, -ran), axis=2)
        else:
            ran = np.random.standard_normal(shape)
        if moment_matching:
            ran = ran - np.mean(ran)
            ran = ran / np.std(ran)
        if shape[0] == 1:
            return ran[0]
        else:
            return ran

In [3]:
snrn = sn_random_numbers((2, 2, 2,), antithetic=False, moment_matching=False, fixed_seed=True)

In [4]:
snrn

array([[[-0.8044583 ,  0.32093155],
        [-0.02548288,  0.64432383]],

       [[-0.30079667,  0.38947455],
        [-0.1074373 , -0.47998308]]])